# Comparación de datos del IPC

In [1]:
# Lectura de librerías
import pandas as pd
from datetime import datetime
from openpyxl import load_workbook
import os

In [31]:
# Leer bases de datos
datos = pd.read_excel("Canasta.xlsx")
boleta = pd.read_excel("Boleta.xlsx")

# Pasar a mayúsculas las columnas
datos.rename(columns={columna: columna.upper() for columna in datos.columns}, inplace=True)
boleta.rename(columns={columna: columna.upper() for columna in boleta.columns}, inplace=True)

# Agregar columna de identificación
datos["IDENTIFICADOR"] = "IPC OFICIAL"
boleta["IDENTIFICADOR"] = "BOLETA"

In [3]:
boleta.head(2)

,CÓDIGO,NOMBRE DE LA VARIEDAD,TAMAÑO,CANTIDAD RECOLECTADA,IDENTIFICADOR
0,01.1.1.2.01.1,HARINA PARA PANQUEQUES,450 GRS,450 GRS,BOLETA
1,01.1.1.2.02.2,PAQUETE DE HARINA DE MAÍZ DE 800 A 1000 GRAMOS,907 GRS,907 GRS,BOLETA


In [4]:
# Limpieza de caracteres atípicos
boleta["TAMAÑO"] = boleta["TAMAÑO"].apply(lambda x: x.replace("RED/10 UNIDADES", "10 UNIDADES"))
boleta["TAMAÑO"] = boleta["TAMAÑO"].apply(lambda x: x.replace("UNIDAD", "1 UNIDAD"))

# Estandarizar formato de datos para poder comparar
datos["DESCRIPCION"] = datos["DESCRIPCION"].str.upper()
datos["UNIDAD DE MEDIDA"] = datos["UNIDAD DE MEDIDA"].str.upper()

# Dividir columna de tamaño en la boleta para que tenga cantidad y unidad de medida por separado
boleta["CANTIDAD"] = boleta["CANTIDAD"].astype("str")
boleta["CANTIDAD"] = boleta["TAMAÑO"].apply(lambda x: x.split()[0] if len(x.split()) > 0 else None)
boleta["UNIDAD DE MEDIDA"] = boleta["TAMAÑO"].apply(lambda x: x.split()[1] if len(x.split()) > 1 else None)

# Hacer columnas de cantidad y unidad de medida para la cantidad recolectada
boleta["CANTIDAD RECOLECTADA"] = boleta["CANTIDAD"].astype("str")
boleta["CANT RECOLECTADA"] = boleta["CANTIDAD RECOLECTADA"].apply(lambda x: x.split()[0] if len(x.split()) > 0 else None)
boleta["UNIDAD RECOLECTADA"] = boleta["CANTIDAD RECOLECTADA"].apply(lambda x: x.split()[1] if len(x.split()) > 1 else None)



# Casteo de columna cantidad
boleta["CANTIDAD"].fillna(-1, inplace=True)
datos["CANTIDAD"].fillna(-1, inplace=True)
boleta["CANTIDAD"] = pd.to_numeric(boleta["CANTIDAD"], errors='coerce')
datos["CANTIDAD"] = pd.to_numeric(datos["CANTIDAD"], errors='coerce')
boleta["CANT RECOLECTADA"].fillna(-1, inplace=True)
boleta["CANT RECOLECTADA"] = pd.to_numeric(boleta["CANT RECOLECTADA"], errors='coerce')


# Renombrar columna de NOMBRE DE LA VARIEDAD
boleta.rename(columns={"NOMBRE DE LA VARIEDAD": "DESCRIPCION", "CÓDIGO": "CODIGO"}, inplace=True)
datos.rename(columns={"CODIGO IPC_2023": "CODIGO"}, inplace=True)

# Agregar columna de cant recolectada y unidad recolectada
datos["CANT RECOLECTADA"] = None
datos["UNIDAD RECOLECTADA"] = None

# Eliminar columna de tamaño
boleta = boleta[["CODIGO", "DESCRIPCION", "CANTIDAD", "UNIDAD DE MEDIDA", "CANT RECOLECTADA", "UNIDAD RECOLECTADA", "IDENTIFICADOR"]]
boleta.drop_duplicates(keep="first", inplace=True)

# Ordenar columnas
datos = datos[["CODIGO", "DESCRIPCION",	"CANTIDAD", "UNIDAD DE MEDIDA", "CANT RECOLECTADA", "UNIDAD RECOLECTADA","IDENTIFICADOR"]]

In [5]:
# Limpieza a cadena quitando tildes, espacios y comas

# Función para limpiar la descripción
def limpiar_descripcion(descripcion):
    # Quitar tildes
    descripcion = descripcion.translate(str.maketrans('áéíóúÁÉÍÓÚ', 'aeiouAEIOU'))
    # Quitar comas
    descripcion = descripcion.replace(',', '')
    # Quitar espacios adicionales
    descripcion = ' '.join(descripcion.split())
    return descripcion

# Aplicar la función a la columna
datos['DESCRIPCION'] = datos['DESCRIPCION'].apply(limpiar_descripcion)
boleta['DESCRIPCION'] = boleta['DESCRIPCION'].apply(limpiar_descripcion)

### Diccionario con unidades de medida

In [6]:
# Crear diccionario con unidades de medida y sus abreviacion
dic_unidades = {'GRAMOS': 'GRS','UNIDAD': 'UNIDADES', 
'MILILITROS': 'ML', 
'UNIDAD ': '', 
'YARDA': '',
'METROS CÚBICOS': '',
'METRO CUADRADO': '',
'SERVICIO': '',
'JUEGO': '',
'METRO': ''}

# Sustituir abreviaciones por su llave en el diccionario
datos["UNIDAD DE MEDIDA"] = datos["UNIDAD DE MEDIDA"].apply(lambda x: dic_unidades.get(x))

## Comparar datos

In [7]:
# Para comparar los datos, se une el dataframe con los códigos en revisión en la boleta y luego se quitan duplicados

# Obtenemos un listado de los códigos que estamos revisando
codigos_en_revision = boleta["CODIGO"].unique().tolist()

# Filtrar datos con los códigos en revisión
datos_comparacion = datos[datos["CODIGO"].isin(codigos_en_revision)]

In [8]:
# Unir dataframes con códigos en revisión
df_revision = pd.concat([datos_comparacion, boleta])

# Los erróneos son aquellos registros que no coinciden con la canasta
df_erroneos = df_revision.drop_duplicates(keep=False, subset=["CODIGO","DESCRIPCION","UNIDAD DE MEDIDA"]).reset_index(drop=True)
# df_erroneos["IDENTIFICADOR"].fillna("BOLETA", inplace=True)

# Agregar datos que no tengan igual cantidad cotizada y tamaño
distintos = boleta[(boleta["CANTIDAD"] != boleta["CANT RECOLECTADA"]) & (~boleta["CANT RECOLECTADA"].isna())].reset_index(drop=True)

C:\Users\cycastañeda\AppData\Local\Temp\ipykernel_16516\1444538546.py:2: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_revision = pd.concat([datos_comparacion, boleta])


In [9]:
boleta[boleta["CODIGO"] == "01.1.3.1.01.2"]

,CODIGO,DESCRIPCION,CANTIDAD,UNIDAD DE MEDIDA,CANT RECOLECTADA,UNIDAD RECOLECTADA,IDENTIFICADOR
30,01.1.3.1.01.2,TILAPIA,1,UNIDAD,NaN,None,BOLETA


In [10]:
def exportar(nombre, df_erroneos, distintos):
    # Obtener la fecha y hora actual
    ahora = datetime.now()

    # Formatear como string
    fecha_hora = ahora.strftime("%H_%M_%S")
    filename = f"Erroneos_{fecha_hora}_de_{nombre}.xlsx"

    # Verificar si el archivo ya existe
    if os.path.exists(filename):
        with pd.ExcelWriter(filename, engine='openpyxl', mode='a') as writer:
            # Si el archivo existe, agrega las hojas nuevas
            df_erroneos.to_excel(writer, sheet_name="Errores con el catalogo")
            distintos.to_excel(writer, sheet_name="Cantidades malas")
    else:
        with pd.ExcelWriter(filename, engine='openpyxl') as writer:
            # Si el archivo no existe, crea uno nuevo
            df_erroneos.to_excel(writer, sheet_name="Errores con el catalogo")
            distintos.to_excel(writer, sheet_name="Cantidades malas")


In [14]:
exportar("Lester", df_erroneos, distintos)

In [15]:
datos[datos["CODIGO"] == "01.1.1.4.02.2"]

,CODIGO,DESCRIPCION,CANTIDAD,UNIDAD DE MEDIDA,CANT RECOLECTADA,UNIDAD RECOLECTADA,IDENTIFICADOR
31,01.1.1.4.02.2,HOJUELAS INSTANTANEA,454.0,GRS,None,None,IPC OFICIAL
